In [1]:
working_dir = "/data/projects/dschaub/ANCA-GN_transcriptomics"
data_dir = "/data/projects/dschaub/ANCA-GN_transcriptomics/data/single-cell/ustekinumab"

In [2]:
%load_ext autoreload
%autoreload 2

import sys
import os

os.chdir(working_dir)

import yaml
import anndata as ad
import matplotlib.pyplot as plt
import seaborn as sns
import mudata as md
import muon as mu
import numpy as np
import scanpy as sc
from matplotlib import gridspec
import scanpy.external as sce
from scipy import sparse
import celltypist
from matplotlib.colors import LinearSegmentedColormap
import matplotlib as mpl

from utils.utils import *
from utils.plotting import *

sc.settings.verbosity = 0
sc.set_figure_params(dpi=80)
sns.set(style="white")

Created a temporary directory at /tmp/tmp2e1yae0u
Writing /tmp/tmp2e1yae0u/_remote_module_non_scriptable.py
Global seed set to 0


## Load data

In [3]:
path = os.path.join(data_dir, "ANCA_ustekinumab_4PK4PB_CD8Teff.h5mu")
mudata = md.read_h5mu(path)
mudata

MuData object with n_obs × n_vars = 3706 × 38528
  obs:	'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_CITE', 'nFeature_CITE', 'frac.mito', 'frac.ribo', 'Feature_Count_ratio', 'CD3_count', 'RNA_snn_res.0.1', 'seurat_clusters', 'RNA_snn_res.0.2', 'RNA_snn_res.0.3', 'RNA_snn_res.0.4', 'RNA_snn_res.0.6', 'nCount_HTO', 'nFeature_HTO', 'HTO_maxID', 'HTO_secondID', 'HTO_margin', 'HTO_classification', 'HTO_classification.global', 'hash.ID', 'patient', 'sample', 'case', 'tissue', 'RNA_snn_res.0.5', 'RNA_snn_res.0.7', 'RNA_snn_res.0.8', 'RNA_snn_res.0.9', 'RNA_snn_res.1', 'cell_type', 'Trm_score1', 'non_Trm_score2', 'CD4_Trm_score3', 'CD4_non_Trm_score4', 'CD8_Trm_score5', 'CD8_non_Trm_score6', 'Proinflam_cytokines_score7', 'RNA_snn_res.0.75'
  obsm:	'X_harmony', 'X_pca', 'X_umap'
  3 modalities
    RNA:	3706 x 38224
      layers:	'counts'
    CITE:	3706 x 302
      layers:	'counts'
    HTO:	3706 x 2
      layers:	'counts'

In [4]:
mudata.mod["rna"] = mudata.mod["RNA"].copy()
mudata.mod["cite"] = mudata.mod["CITE"].copy()
del mudata.mod["RNA"]
del mudata.mod["CITE"]
del mudata.mod["HTO"]
mudata.update()
mudata

MuData object with n_obs × n_vars = 3706 × 38526
  obs:	'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_CITE', 'nFeature_CITE', 'frac.mito', 'frac.ribo', 'Feature_Count_ratio', 'CD3_count', 'RNA_snn_res.0.1', 'seurat_clusters', 'RNA_snn_res.0.2', 'RNA_snn_res.0.3', 'RNA_snn_res.0.4', 'RNA_snn_res.0.6', 'nCount_HTO', 'nFeature_HTO', 'HTO_maxID', 'HTO_secondID', 'HTO_margin', 'HTO_classification', 'HTO_classification.global', 'hash.ID', 'patient', 'sample', 'case', 'tissue', 'RNA_snn_res.0.5', 'RNA_snn_res.0.7', 'RNA_snn_res.0.8', 'RNA_snn_res.0.9', 'RNA_snn_res.1', 'cell_type', 'Trm_score1', 'non_Trm_score2', 'CD4_Trm_score3', 'CD4_non_Trm_score4', 'CD8_Trm_score5', 'CD8_non_Trm_score6', 'Proinflam_cytokines_score7', 'RNA_snn_res.0.75'
  obsm:	'X_harmony', 'X_pca', 'X_umap', 'RNA', 'CITE', 'HTO'
  varm:	'RNA', 'CITE', 'HTO'
  2 modalities
    rna:	3706 x 38224
      layers:	'counts'
    cite:	3706 x 302
      layers:	'counts'

## Prepare data

In [ ]:
obs = mudata.obs[
    ["orig.ident", "RNA_snn_res.0.75", "patient", "sample", "case", "tissue"]
].copy()

In [6]:
obsm = {
    "X_umap": mudata.obsm["X_umap"],
    "X_pca": mudata.obsm["X_pca"],
    "X_harmony": mudata.obsm["X_harmony"],
}
mod_rna = mudata.mod["rna"]
mod_cite = mudata.mod["cite"]

mod_rna.obs = obs.copy()
mod_cite.obs = obs.copy()

mod_rna.obsm = obsm.copy()
mod_cite.obsm = obsm.copy()

mod_rna.X = mod_rna.layers["counts"].copy()
mod_cite.X = mod_cite.layers["counts"].copy()

log_normalize(mod_rna)
protein_clr(mod_cite)

mudata = md.MuData({"rna": mod_rna, "cite": mod_cite})
mudata

MuData object with n_obs × n_vars = 3706 × 38526
  2 modalities
    rna:	3706 x 38224
      obs:	'orig.ident', 'RNA_snn_res.0.75', 'patient', 'sample', 'case', 'tissue'
      uns:	'log1p'
      obsm:	'X_umap', 'X_pca', 'X_harmony'
      layers:	'counts', 'log1p'
    cite:	3706 x 302
      obs:	'orig.ident', 'RNA_snn_res.0.75', 'patient', 'sample', 'case', 'tissue'
      obsm:	'X_umap', 'X_pca', 'X_harmony'
      layers:	'counts', 'clr'

In [7]:
mudata.obs

,rna:orig.ident,rna:RNA_snn_res.0.75,rna:patient,rna:sample,rna:case,rna:tissue,cite:orig.ident,cite:RNA_snn_res.0.75,cite:patient,cite:sample,cite:case,cite:tissue
T003K_AAACCTGCAGCCTTTC,T003,1,T003,T003K,P,K,T003,1,T003,T003K,P,K
T003K_AAACGGGGTCTTGTCC,T003,3,T003,T003K,P,K,T003,3,T003,T003K,P,K
T003K_AAAGATGAGGACTGGT,T003,1,T003,T003K,P,K,T003,1,T003,T003K,P,K
T003K_AAAGATGCACGAGGTA,T003,3,T003,T003K,P,K,T003,3,T003,T003K,P,K
T003K_AAAGATGTCCTTGACC,T003,1,T003,T003K,P,K,T003,1,T003,T003K,P,K
...,...,...,...,...,...,...,...,...,...,...,...,...
T004B_TTAGTTCCATCGACGC-1,T004B,0,T004,T004B,P,B,T004B,0,T004,T004B,P,B
T004B_TTCGAAGGTAGCGATG-1,T004B,0,T004,T004B,P,B,T004B,0,T004,T004B,P,B
T004B_TTCTCCTCATCAGTAC-1,T004B,0,T004,T004B,P,B,T004B,0,T004,T004B,P,B
T004B_TTGTAGGAGGGAGTAA-1,T004B,0,T004,T004B,P,B,T004B,0,T004,T004B,P,B


In [8]:
mod_rna = mudata.mod["rna"]
mod_cite = mudata.mod["cite"]

In [9]:
save_path = os.path.join(data_dir, "ANCA_ustekinumab_4PK4PB_CD8Teff_processed.h5mu")
mudata.write_h5mu(save_path)

... storing 'orig.ident' as categorical
... storing 'patient' as categorical
... storing 'sample' as categorical
... storing 'case' as categorical
... storing 'tissue' as categorical
... storing 'orig.ident' as categorical
... storing 'patient' as categorical
... storing 'sample' as categorical
... storing 'case' as categorical
... storing 'tissue' as categorical
